# Transfer Learning 

<tr>
    <td><img src="images/imagenet_challenge.png" alt="drawing" style="width:500px;" align="left"/></td>
    <td><img src="images/screenshot.png" alt="drawing" style="width:500px;" align="left"/></td>
</tr>

## Classification Problem - Emergency vs Non-emergency Vehicle Classification

<img src='images/emergency_vs_non.png'>


[Source](https://www.slideshare.net/xavigiro/image-classification-on-imagenet-d1l4-2017-upc-deep-learning-for-computer-vision/)

Table of Contents
1. <font color='blue'>Import neccessary libraries
2. Load the data</font>
3. <font color='#999900'>Preprocess the data</font>
4. <font color='red'>Load weights of pre-trained model
5. Fine tune the model for the current problem</font>
6. <font color='blue'>Get predictions</font>

## 1. Import neccessary libraries

Here we will import the required libraries and functions.

In [1]:
# import necessary libraries for plotting images
import numpy as np
import matplotlib.pyplot as plt
# plot in jupyter notebook itself
%matplotlib inline
from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from skimage.transform import resize

from keras.utils import to_categorical
#used to preprocess data according to VGG16
from keras.applications.vgg16 import preprocess_input
#for instantiating the model and loading the weights and biases
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, InputLayer

Using TensorFlow backend.


## 2. Load the data

In [3]:
#reading the csv file containing data labels
path ="./../CV101/datasets/emergency_classification/"
data = pd.read_csv((path+"emergency_classification.csv"))
#data = pd.read_csv('emergency/train_SOaYf6m/train.csv')

In [4]:
data.head()

,image_names,emergency_or_not
0,0.jpg,1
1,1.jpg,1
2,2.jpg,1
3,3.jpg,1
4,4.jpg,1


In [12]:
# create an empty python list
X = []

# go through all the image locations one by one
for img_name in data.image_names:
    # read the image from location
    img = plt.imread(path+"/images/"+img_name)
    img = img.astype(np.float16)

    # pile it one over the other
    X.append(img)
    
# convert this python list to a single numpy array
X = np.array(X)

In [27]:
X.shape

(2352, 224, 224, 3)

In [13]:
#getting the labels for images
y = data.emergency_or_not.values

In [14]:
#converting label to categorical i.e instead of 0/1 labels we have 2 columns emergency and non-emergency ,
#with only one of them is true for every image
y = to_categorical(y)

In [30]:
X.shape

(2352, 224, 224, 3)

## 3. Pre-Process Data

Steps : 
1. Pre-process the data as per model's requirement
2. Prepare training and validation set

In [15]:
#show maximum and minimum values for the image array
X.min(), X.max()

(0.0, 255.0)

In [16]:
#preprocess input images accordiing to requirements of VGG16 model
X = preprocess_input(X, mode='tf')

In [17]:
#print minimum and maximum values present in the array
X.min(), X.max()

(-1.0, 1.0)

In [18]:
# splitting the dataset into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

## 4. Load weights of pretrained model

In [19]:
# creating model with pre trained imagenet weights
base_model = VGG16(weights='imagenet')

In [20]:
#shows model summary
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [21]:
# creating a VGG16 model with imagenet pretrained weights , accepting input of shape (224,224,3) and remove
#also remove the final layers from model(include_top= False)
base_model = VGG16(weights='imagenet', input_shape=(224, 224, 3), include_top=False)

In [22]:
# show model summary
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

## 5. Fine tune the model for the current problem
Steps:-
1. Extract features
2. Flatten the data
3. Rescale features
4. Create a Neural Network Model
5. Compile the model
6. Train and Validate the model

In [25]:
X_train.shape

(1646, 224, 224, 3)

In [26]:
X_valid.shape

(706, 224, 224, 3)

In [24]:
# extract features using the pretrained VGG16 model
# for training set
base_model_pred = base_model.predict(X_train)


In [23]:
#for validation set
base_model_pred_valid = base_model.predict(X_valid)

In [28]:
#show shape of predictions
base_model_pred.shape

(1646, 7, 7, 512)

In [31]:
# flattening the model output to one dimension for every sample of training set
base_model_pred = base_model_pred.reshape(1646, 7*7*512)

In [32]:
base_model_pred.shape

(1646, 25088)

In [18]:
base_model_pred.shape

(1152, 25088)

In [19]:
base_model_pred_valid.shape

(494, 7, 7, 512)

In [20]:
# flattening the model output to one dimension for every sample of validation set
base_model_pred_valid = base_model_pred_valid.reshape(494, 7*7*512)

In [21]:
base_model_pred_valid.shape

(494, 25088)

In [22]:
# checking the min and max of the extracted features
base_model_pred.min(), base_model_pred.max()

(0.0, 11.164914)

In [23]:
#get maximum value from generated features
max_val = base_model_pred.max()

In [24]:
#normalizing features generated from the VGG16 model to [0,1]
base_model_pred = base_model_pred / max_val
base_model_pred_valid = base_model_pred_valid / max_val
base_model_pred.min(), base_model_pred.max()

(0.0, 1.0)

In [25]:
#create a sequential model 
model = Sequential()
# add input layer to the model that accepts input of shape 7*7*512
model.add(InputLayer((7*7*512, )))
# add fully connected layer with 1024 neurons and relu activation
model.add(Dense(units=1024, activation='relu'))
# add fully connected layer with 2 neurons and relu activation
model.add(Dense(units=2, activation='softmax'))

In [26]:
# compile the model
model.compile(optimizer='sgd', metrics=['accuracy'], loss='categorical_crossentropy')

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2050      
Total params: 25,693,186
Trainable params: 25,693,186
Non-trainable params: 0
_________________________________________________________________


In [28]:
# train model using features generated from VGG16 model
model.fit(base_model_pred, y_train, epochs=100, validation_data=(base_model_pred_valid, y_valid))

Train on 1152 samples, validate on 494 samples
Epoch 1/100
1152/1152 [==============================] - 1s 683us/step - loss: 0.6501 - acc: 0.6050 - val_loss: 0.6275 - val_acc: 0.6255
Epoch 2/100
1152/1152 [==============================] - 0s 291us/step - loss: 0.5928 - acc: 0.6875 - val_loss: 0.5787 - val_acc: 0.7551
Epoch 3/100
1152/1152 [==============================] - 0s 286us/step - loss: 0.5466 - acc: 0.7865 - val_loss: 0.5482 - val_acc: 0.7287
Epoch 4/100
1152/1152 [==============================] - 0s 293us/step - loss: 0.5055 - acc: 0.8030 - val_loss: 0.5030 - val_acc: 0.8097
Epoch 5/100
1152/1152 [==============================] - 0s 300us/step - loss: 0.4685 - acc: 0.8438 - val_loss: 0.4866 - val_acc: 0.7814
Epoch 6/100
1152/1152 [==============================] - 0s 296us/step - loss: 0.4419 - acc: 0.8368 - val_loss: 0.4459 - val_acc: 0.8097
Epoch 7/100
1152/1152 [==============================] - 0s 288us/step - loss: 0.4138 - acc: 0.8568 - val_loss: 0.4420 - val_acc: 0

1152/1152 [==============================] - 0s 302us/step - loss: 0.0909 - acc: 0.9818 - val_loss: 0.1869 - val_acc: 0.9190
Epoch 61/100
1152/1152 [==============================] - 0s 299us/step - loss: 0.0878 - acc: 0.9809 - val_loss: 0.1863 - val_acc: 0.9190
Epoch 62/100
1152/1152 [==============================] - 0s 286us/step - loss: 0.0855 - acc: 0.9826 - val_loss: 0.1951 - val_acc: 0.9190
Epoch 63/100
1152/1152 [==============================] - 0s 288us/step - loss: 0.0847 - acc: 0.9826 - val_loss: 0.1856 - val_acc: 0.9170
Epoch 64/100
1152/1152 [==============================] - 0s 299us/step - loss: 0.0816 - acc: 0.9835 - val_loss: 0.1828 - val_acc: 0.9170
Epoch 65/100
1152/1152 [==============================] - 0s 301us/step - loss: 0.0798 - acc: 0.9844 - val_loss: 0.1823 - val_acc: 0.9170
Epoch 66/100
1152/1152 [==============================] - 0s 295us/step - loss: 0.0792 - acc: 0.9870 - val_loss: 0.1843 - val_acc: 0.9190
Epoch 67/100
1152/1152 [=======================

## 6. Get Predictions

In [29]:
# get predictions
predictions = model.predict_classes(base_model_pred_valid)
#show predictions
predictions

array([1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Summary of the video

1. <font color='blue'>Import neccessary libraries
2. Load the data</font>
3. <font color='#999900'>Preprocess the data</font>
4. <font color='red'>Load weights of pre-trained model
5. Fine tune the model for the current problem</font>
6. <font color='blue'>Get predictions</font>